# Rationale

Often, when studying Python, you are first introduced to the syntax of function signatures by learning about positional-or-keyword (here called "mixed") parameters. Then, positional-only and keyword-only parameters are usually explained.

Probably at some point, parameters of arbitrary lengths (`*args` and `**kwargs`) are explained too. Often one is familiar with both of these two syntaxes:

```python
def func(pos1, ..., posn, /, mxd1, ..., mxdn, *, key1, ..., keyn):
    ...
```

and

```python
def func(mxd1, ..., mxdn, *args, **kwargs):
    ...
```

but without knowing the general case that lies behind them.

__The purpose of this notebook is to provide concrete examples and deduce a general syntax that is much more intuitive than the specific individual syntaxes__.

# Conventions

The following conventions will be used:
- `pos<n>` for positional-only parameters, e.g., `pos1`, `pos2`, etc.
- `key<n>` for keyword-only parameters, e.g., `key1`, `key2`, etc.
- `mxd<n>` for positional-or-keyword parameters, i.e., the default ones when in the signature it is not specified neither `/` nor `*`, e.g., `mxd1`, `mxd2`, etc.

The parameters are those defined in the function signature, while the arguments are the values passed in the function call.

In [1]:
def func(par1):
    print(f'par1: {par1}')


arg1 = 'a'
func(arg1)

par1: a


# TL;DR

Here's the most general syntax for function signatures:

```python
def func([<positional-only parameters>,] [/,] [<mixed parameters>,] [*[args],] [<keyword-only parameters>,] [**kwargs]):
```

`/` and `*` are the delimiters for positional-only and keyword-only parameters, respectively. However, the `*` delimiter is also used for specifying `args` (it would be better to use `<args>` as a placeholder, but since the name `args` is commonly used as a convention, we'll follow that).

This means that in the signature, you can use the `*` delimiter just once, and if you'd like to include `args` in addition to keyword-only parameters (or vice versa), you should write `*args`.

The intuitive rule for remembering this is: **you can only use the `*` delimiter once, either alone or immediately followed by `args`, and everything that follows `*` is a keyword-only parameter (except for `**kwargs`)**.

Now that this has been clarified: you can also specify default values, which are something totally different and often confused with the concept of keyword-only parameters.

Here, the main rule is: **every kind of parameter can have a default value defined in the signature. If you choose to assign it, then in the function signature there must be a point where on the left you only have parameters without default values, and on the right you only have parameters with default values *or* that are one of the following: `*args`, keyword-only, `**kwargs`.**

As a direct implication of this, keyword-only parameters can have default values or not, without any specific ordering requirements.

# Parameters

## Mixed Parameters

By default in Python, unless otherwise specified, all parameters are mixed parameters. This means that they can behave as both positional and keyword parameters:

In [2]:
def func(mxd1, mxd2):
    print(f'mxd1: {mxd1}\nmxd2: {mxd2}')

In [3]:
func('a', 'b')

mxd1: a
mxd2: b


Passing `'a'` and `'b'` will assign them to `mxd1` and `mxd2`, respectively. Both parameters are here used as positional parameters.

Since they are mixed parameters, you can also use keyword arguments when passing values in the function call:

In [4]:
func(mxd1='a', mxd2='b')

mxd1: a
mxd2: b


Using them as keyword arguments also allows you to pass them in any order:

In [5]:
func(mxd2='b', mxd1='a')

mxd1: a
mxd2: b


You can also mix positional and keyword arguments:

In [6]:
func('a', mxd2='b')

mxd1: a
mxd2: b


However, all keyword arguments **must** be placed to the right of positional arguments. If not, Python will raise a `SyntaxError` exception:

In [7]:
try:
    exec("func(mxd1='a', 'b')")
except SyntaxError as e:
    print(e)

positional argument follows keyword argument (<string>, line 1)


## Positional-only Parameters

If you want to force any number of parameters to be positional-only, write them separated by commas, followed by a trailing `/`, as if it were an additional parameter:

In [8]:
def func(pos1, pos2, /):
    print(f'pos1: {pos1}\npos2: {pos2}')

In [9]:
func('a', 'b')

pos1: a
pos2: b


Since they are positional-only parameters, attempting to pass them as keyword arguments will result in Python raising a `TypeError` exception:

In [10]:
def func(pos1, pos2, /):
    print(f'pos1: {pos1}\npos2: {pos2}')

In [11]:
try:
    func(pos1='a', pos2='b')
except TypeError as e:
    print(e)

func() got some positional-only arguments passed as keyword arguments: 'pos1, pos2'


After the `/` delimiter, you can specify any number of mixed parameters.

In [12]:
def func(pos1, pos2, /, mxd1):
    print(f'pos1: {pos1}\npos2: {pos2}\nmxd1: {mxd1}')

In [13]:
func('a', 'b', 'c')

pos1: a
pos2: b
mxd1: c


In [14]:
func('a', 'b', mxd1='c')

pos1: a
pos2: b
mxd1: c


## Keyword-only Parameters

If you want to force any number of parameters to be keyword-only, write them separated by commas, preceded by a `*`, as if it were a parameter:

In [15]:
def func(*, key1, key2):
    print(f'key1: {key1}\nkey2: {key2}')

In [16]:
func(key1='a', key2='b')

key1: a
key2: b


Since they are keyword-only arguments, you can pass them in any order:

In [17]:
func(key2='b', key1='a')

key1: a
key2: b


Since they are keyword-only parameters, attempting to pass them as positional arguments will result in Python raising a `TypeError` exception:

In [18]:
try:
    func('a', key2='b')
except TypeError as e:
    print(e)

func() takes 0 positional arguments but 1 positional argument (and 1 keyword-only argument) were given


Before the `*` delimiter, you can specify any number of mixed parameters:

In [19]:
def func(mxd1, *, key1, key2):
    print(f'key1: {key1}\nkey2: {key2}\nmxd1: {mxd1}')

In [20]:
func('a', key1='b', key2='c')

key1: b
key2: c
mxd1: a


In [21]:
func(mxd1='a', key1='b', key2='c')

key1: b
key2: c
mxd1: a


If you use the mixed parameter(s) as keyword arguments, you can pass all the arguments in any order:

In [22]:
func(key1='b', mxd1='a', key2='c')

key1: b
key2: c
mxd1: a


##  Tying Them All Together

Here's a list of examples that summarize what we've seen so far:

In [23]:
def func(pos1, /, mxd1, *, key1):
    print(f'pos1: {pos1}\nmxd1: {mxd1}\nkey1: {key1}')

In [24]:
func('a', 'b', key1='c')

pos1: a
mxd1: b
key1: c


In [25]:
func('a', mxd1='b', key1='c')

pos1: a
mxd1: b
key1: c


In [26]:
func('a', key1='c', mxd1='b')

pos1: a
mxd1: b
key1: c


Mixed parameters are not mandatory; you can have just positional-only and keyword-only parameters using the following syntax:

In [27]:
def func(pos1, /, *, key1):
    print(f'pos1: {pos1}\nkey1: {key1}')

In [28]:
func('a', key1='b')

pos1: a
key1: b


All the above rules still apply. Therefore, it's not possible to pass a keyword argument to a positional-only parameter, and vice versa. Otherwise, Python will raise a `TypeError` exception:

In [29]:
try:
    func(pos1='a', key1='b')
except TypeError as e:
    print(e)

func() got some positional-only arguments passed as keyword arguments: 'pos1'


In [30]:
try:
    func('a', 'b')
except TypeError as e:
    print(e)

func() takes 1 positional argument but 2 were given


It's also not possible to mix up the order of positional and keyword arguments in the function call. In this case, Python will raise a `SyntaxError` since the arguments, though used correctly, are in the wrong order:

In [31]:
try:
    exec("func(key1='b', 'a')")
except SyntaxError as e:
    print(e)

positional argument follows keyword argument (<string>, line 1)


## `*args` and `**kwargs`

### `*args`

The name of the parameter following the `*` represents the tuple that will collect any number of positional arguments. Here, we use `args` because it is the conventional name for this particular parameter.

In [32]:
def func(*args):
    print(f"args: {args}\nargs' type is: {type(args)}")

In [33]:
func('a', 'b', 'c')

args: ('a', 'b', 'c')
args' type is: <class 'tuple'>


You can pass any type of object, and it will be collected as an element of `args`:

In [34]:
def func(*args):
    print(f'args: {args}')

In [35]:
func(
    'a',
    'b',
    'c',
    ('d', 'e', 'f'),
    ['g', 'h', 'i'],
    {'j', 'k', 'l'},
    {'m': 1, 'n': 1, 'o': 1},
)

args: ('a', 'b', 'c', ('d', 'e', 'f'), ['g', 'h', 'i'], {'j', 'k', 'l'}, {'m': 1, 'n': 1, 'o': 1})


Every parameter following the `*` is, by default, a keyword-only parameter:

In [36]:
def func(*args, key1, key2):
    print(f'args: {args}\nkey1: {key1}\nkey2: {key2}')

In [37]:
func('a', 'b', 'c', key1='d', key2='e')

args: ('a', 'b', 'c')
key1: d
key2: e


You can't use the `*` more than once. If you use it in `*args`, it is already functioning as a delimiter. If you use it more than once, Python will raise a `SyntaxError`:

In [38]:
try:
    exec(
        """
def func(*args, *, key1, key2):
    print(f'args: {args}\\nkey1: {key1}\\nkey2: {key2}')
    """
    )
except SyntaxError as e:
    print(e)

invalid syntax (<string>, line 2)


For the same reason, everything before the `*` in `*args` would be considered a mixed parameter:

In [39]:
def func(mxd1, *args):
    print(f'mxd1: {mxd1}\nargs: {args}')

In [40]:
func('a', 'b', 'c')

mxd1: a
args: ('b', 'c')


In [41]:
def func(mxd1, *args, key1, key2):
    print(f'mxd1: {mxd1}\nargs: {args}\nkey1: {key1}\nkey2: {key2}')

In [42]:
func('a', 'b', 'c', key1='d', key2='e')

mxd1: a
args: ('b', 'c')
key1: d
key2: e


If, in the function call, you pass `mxd1` as a keyword argument, Python will raise a `SyntaxError` because at least one positional argument is following a keyword argument:

In [43]:
try:
    exec("func(mxd1='a', 'b', 'c', key1='d', key2='e')")
except SyntaxError as e:
    print(e)

positional argument follows keyword argument (<string>, line 1)


If you rearrange the arguments to have all the keyword arguments after the positional arguments, in this specific case Python will still raise a `TypeError` exception. This is because `'b'` is assigned to `mxd1`, `*args` catches `'c'`, and `'a'` is assigned again to `mxd1`:

In [44]:
try:
    func('b', 'c', mxd1='a', key1='d', key2='e')
except TypeError as e:
    print(e)

func() got multiple values for argument 'mxd1'


Apparently, using a mixed parameter before `*args` effectively forces you to use it as a positional-only parameter. This is because `*args` is designed to accept a list of positional arguments, meaning you're not allowed to use the mixed parameter as a keyword parameter.

Hence, using a mixed parameter before `*args` is functionally equivalent to using a positional-only parameter.

In [45]:
def func(pos1, /, *args, key1, key2):
    print(f'pos1: {pos1}\nargs: {args}\nkey1: {key1}\nkey2: {key2}')

In [46]:
func('a', 'b', 'c', key1='d', key2='e')

pos1: a
args: ('b', 'c')
key1: d
key2: e


### `**kwargs`

The name of the parameter following the `**` represents the dictionary that will collect any number of keyword arguments. Here, we use `kwargs` because it is the conventional name for this particular parameter.

`**kwargs` is **always** the last parameter in the signature.

In [47]:
def func(**kwargs):
    for k, v in kwargs.items():
        print(f'{k}: {v}')

    print(f"kwargs' type is: {type(kwargs)}")

In [48]:
func(key1='a', key2='b', key3='c')

key1: a
key2: b
key3: c
kwargs' type is: <class 'dict'>


Since `kwargs` is a dictionary, you can pass any object type as a value:

In [49]:
def func(**kwargs):
    for k, v in kwargs.items():
        print(f'{k}: {v}')

In [50]:
func(
    key1='a',
    key2='b',
    key3='c',
    key4=('d', 'e', 'f'),
    key5=['g', 'h', 'i'],
    key6={'j', 'k', 'l'},
    key7={'m': 1, 'n': 1, 'o': 1},
)

key1: a
key2: b
key3: c
key4: ('d', 'e', 'f')
key5: ['g', 'h', 'i']
key6: {'j', 'k', 'l'}
key7: {'m': 1, 'n': 1, 'o': 1}


Before `**kwargs`, you can include `*args`, and as expected, `*args` will catch all the available positional arguments, while `**kwargs` will catch all the available keyword arguments:

In [51]:
def func(*args, **kwargs):
    print(f'args: {args}')
    for k, v in kwargs.items():
        print(f'{k}: {v}')

In [52]:
func('a', 'b', 'c', key1='d', key2='e', key3='f')

args: ('a', 'b', 'c')
key1: d
key2: e
key3: f


Everything between `*args` and `**kwargs` **must** be a keyword-only parameter:

In [53]:
def func(*args, key1, key2, **kwargs):
    print(f'args: {args}\nkey1: {key1}\nkey2: {key2}\nkwargs: {kwargs}')

In [54]:
func('a', 'b', 'c', key1='d', key2='e', key3='f', key4='g')

args: ('a', 'b', 'c')
key1: d
key2: e
kwargs: {'key3': 'f', 'key4': 'g'}


As always, `*args` causes everything before it to be treated as positional-only parameters, even if they are mixed. The function call will raise a `SyntaxError` because there is a positional argument following a keyword argument:

In [55]:
def func(mxd1, *args, key1, **kwargs):
    print(f'mxd1: {mxd1}\nargs: {args}\nkey1: {key1}\nkwargs: {kwargs}')

In [56]:
try:
    exec(
        "func(mxd1='a', 'b', 'c', 'd', key1='d', key2='e', key3='f', key4='g')"
    )
except SyntaxError as e:
    print(e)

positional argument follows keyword argument (<string>, line 1)


Here we have three parameters, one of each kind. Note that, unlike `*args`, `**kwargs` doesn't impose any behavior on the parameters preceding it. `**kwargs` will catch all the keyword arguments that are not `key1`:

In [57]:
def func(pos1, /, mxd1, *, key1, **kwargs):
    print(f'pos1: {pos1}\nmxd1: {mxd1}\nkey1: {key1}\nkwargs: {kwargs}')

In [58]:
func('a', 'b', key1='c', key2='d', key3='e')

pos1: a
mxd1: b
key1: c
kwargs: {'key2': 'd', 'key3': 'e'}


In [59]:
func('a', mxd1='b', key1='c', key2='d', key3='e')

pos1: a
mxd1: b
key1: c
kwargs: {'key2': 'd', 'key3': 'e'}


In [60]:
func('a', key1='c', mxd1='b', key2='d', key3='e')

pos1: a
mxd1: b
key1: c
kwargs: {'key2': 'd', 'key3': 'e'}


After the last positional-only parameter, mixed parameters used as keywords and keyword-only parameters can be used in any order. Any keywords that are not defined in the function signature will be caught by `**kwargs`, even if they appear before the mixed parameters in the function call:

In [61]:
func('a', key1='c', key2='d', mxd1='b', key3='e')

pos1: a
mxd1: b
key1: c
kwargs: {'key2': 'd', 'key3': 'e'}


# Default Values

As stated in the TL;DR section: **every kind of parameter can have a default value defined in the signature. If you choose to assign it, then in the function signature there must be a point where on the left you only have parameters without default values, and on the right you only have parameters with default values or those that are one of the following: `*args`, keyword-only, `**kwargs`**.

First, here's an example without default values:

In [62]:
def func(pos1, pos2, /, mxd1, mxd2, *args, key1, key2, **kwargs):
    print(
        f'pos1: {pos1}\npos2: {pos2}\nmxd1: {mxd1}\nmxd2: {mxd2}\n'
        f'args: {args}\nkey1: {key1}\nkey2: {key2}\nkwargs: {kwargs}'
    )

In [63]:
func('a', 'b', 'c', 'd', 'e', 'f', key1='g', key2='h', key3='i', key4='j')

pos1: a
pos2: b
mxd1: c
mxd2: d
args: ('e', 'f')
key1: g
key2: h
kwargs: {'key3': 'i', 'key4': 'j'}


Here's a keyword-only parameter with a default value. Note that after `key1`, another keyword-only parameter, `key2`, follows without any default value. This is possible because keyword-only parameters are not subject to the constraint of having all the parameters with default values on the right side:

In [64]:
def func(pos1, pos2, /, mxd1, mxd2, *args, key1=None, key2, **kwargs):
    print(
        f'pos1: {pos1}\npos2: {pos2}\nmxd1: {mxd1}\nmxd2: {mxd2}\n'
        f'args: {args}\nkey1: {key1}\nkey2: {key2}\nkwargs: {kwargs}'
    )

In [65]:
func('a', 'b', 'c', 'd', 'e', 'f', key2='g', key3='h', key4='i')

pos1: a
pos2: b
mxd1: c
mxd2: d
args: ('e', 'f')
key1: None
key2: g
kwargs: {'key3': 'h', 'key4': 'i'}


Here, `mxd2` has a default value but still respects the constraint of having only `*args`, keyword-only, or `**kwargs` on the right:

In [66]:
def func(pos1, pos2, /, mxd1, mxd2=None, *args, key1=None, key2, **kwargs):
    print(
        f'pos1: {pos1}\npos2: {pos2}\nmxd1: {mxd1}\nmxd2: {mxd2}\n'
        f'args: {args}\nkey1: {key1}\nkey2: {key2}\nkwargs: {kwargs}'
    )

In [67]:
func('a', 'b', 'c', 'd', 'e', 'f', key2='g', key3='h', key4='i')

pos1: a
pos2: b
mxd1: c
mxd2: d
args: ('e', 'f')
key1: None
key2: g
kwargs: {'key3': 'h', 'key4': 'i'}


If a non-keyword parameter without a default value is placed to the right of one that has a default value, Python will raise a `SyntaxError`:

In [68]:
try:
    exec(
        """
def func(pos1, pos2, /, mxd1=None, mxd2, *, key1=None, key2, **kwargs):
    print(f'pos1: {pos1}\npos2: {pos2}\nmxd1: {mxd1}\nmxd2: {mxd2}\n' \
    f'key1: {key1}\nkey2: {key2}\nkwargs: {kwargs}')
"""
    )
except SyntaxError as e:
    print(e)

unterminated string literal (detected at line 3) (<string>, line 3)


Here, all the parameters, except for the first positional-only one, have a default value. This is consistent with the rule mentioned above:

In [69]:
def func(
    pos1, pos2=None, /, mxd1=None, mxd2=None, *, key1=None, key2, **kwargs
):
    print(
        f'pos1: {pos1}\npos2: {pos2}\nmxd1: {mxd1}\nmxd2: {mxd2}\n'
        f'key1: {key1}\nkey2: {key2}\nkwargs: {kwargs}'
    )

In [70]:
func('a', key1='c', key2='d', key3='e', key4='f')

pos1: a
pos2: None
mxd1: None
mxd2: None
key1: c
key2: d
kwargs: {'key3': 'e', 'key4': 'f'}


# Bonus

Consider the syntax of a `lambda` expression:

```python
lambda <parameters-list> : <return-value>
```

The `<parameters-list>` will behave exactly like the parameter list in a standard function. Everything shown so far still applies to the parameter list of a `lambda` expression.

Since the `lambda` expression, when evaluated, produces a function, if you wrap the expression in `()`, you can make a function call on the fly:

```python
(<lambda-expression>)(<arguments-list>)
```

Again, the list of arguments is like a standard one when performing a function call, and all the same rules seen so far still apply.

This means that the following `lambda` expression, used to make a function call on the fly, is perfectly fine:

In [71]:
(
    lambda pos1, /, mxd1, *args, key1, **kwargs: (
        pos1,
        mxd1,
        args,
        key1,
        kwargs,
    )
)('a', 'b', 'c', 'd', key1='e', key2='f', key3='g')

('a', 'b', ('c', 'd'), 'e', {'key2': 'f', 'key3': 'g'})